In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import os

In [2]:
urls = {
    'Anxiety': 'https://medium.com/tag/anxiety/archive/{0}/{1:02d}/{2:02d}',
    'AnxietyMonths' : 'https://medium.com/tag/anxiety/archive/{0}/{1:02d}'
}

In [3]:
def is_leap(year):
    if year % 4 != 0:
        return False
    elif year % 100 != 0:
        return True
    elif year % 400 != 0:
        return False
    else:
        return True
    
def convert_day(day, year):
    month_days = [31, 29 if is_leap(year) else 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    m = 0
    d = 0
    while day > 0:
        d = day
        day -= month_days[m]
        m += 1
    return (m, d)

In [4]:
year = 2020
selected_days = [i for i in range(1, 367 if is_leap(year) else 366)]

In [5]:
img_dir = 'images'
if not os.path.exists(img_dir):
    os.mkdir(img_dir)

In [6]:
data = []
article_id = 0
i = 0
n = len(selected_days)
if int(year) <= 2015:
    url = 'https://medium.com/tag/anxiety/archive/{0}/{1:02d}'
    for month in range(1,13):
        date = '{0}-{1:02d}'.format(year, month)
        print(url.format(year, month))
        response = requests.get(url.format(year, month), allow_redirects=True)
        if response.url.startswith(url.format(year, month)):
            page = response.content
            soup = BeautifulSoup(page, 'html.parser')
            articles = soup.find_all("div", class_="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls")
            for article in articles:
                title = article.find("h3", class_="graf--title")
                if title is None:
                    continue
                title = title.contents[0]
                article_id += 1
                article_url = article.find_all("a")[3]['href'].split('?')[0]

                data.append([article_id, article_url, title, date])
else:
    url = 'https://medium.com/tag/anxiety/archive/{0}/{1:02d}/{2:02d}'
    for d in selected_days:
        i += 1
        month, day = convert_day(d, year)
        date = '{0}-{1:02d}-{2:02d}'.format(year, month, day)
        print(url.format(year, month, day))
        response = requests.get(url.format(year, month, day), allow_redirects=True)
        if not response.url.startswith(url.format(year, month, day)):
            continue
        page = response.content
        soup = BeautifulSoup(page, 'html.parser')
        articles = soup.find_all("div", class_="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls")
        for article in articles:
            title = article.find("h3", class_="graf--title")
            if title is None:
                continue
            #title = BeautifulSoup(title.contents[0])
            only_title = title.get_text()
            article_id += 1
            article_url = article.find_all("a")[3]['href'].split('?')[0]

            data.append([article_id, article_url, only_title, date])
        

https://medium.com/tag/anxiety/archive/2020/01/01
https://medium.com/tag/anxiety/archive/2020/01/02
https://medium.com/tag/anxiety/archive/2020/01/03
https://medium.com/tag/anxiety/archive/2020/01/04
https://medium.com/tag/anxiety/archive/2020/01/05
https://medium.com/tag/anxiety/archive/2020/01/06
https://medium.com/tag/anxiety/archive/2020/01/07
https://medium.com/tag/anxiety/archive/2020/01/08
https://medium.com/tag/anxiety/archive/2020/01/09
https://medium.com/tag/anxiety/archive/2020/01/10
https://medium.com/tag/anxiety/archive/2020/01/11
https://medium.com/tag/anxiety/archive/2020/01/12
https://medium.com/tag/anxiety/archive/2020/01/13
https://medium.com/tag/anxiety/archive/2020/01/14
https://medium.com/tag/anxiety/archive/2020/01/15
https://medium.com/tag/anxiety/archive/2020/01/16
https://medium.com/tag/anxiety/archive/2020/01/17
https://medium.com/tag/anxiety/archive/2020/01/18
https://medium.com/tag/anxiety/archive/2020/01/19
https://medium.com/tag/anxiety/archive/2020/01/20


https://medium.com/tag/anxiety/archive/2020/06/13
https://medium.com/tag/anxiety/archive/2020/06/14
https://medium.com/tag/anxiety/archive/2020/06/15
https://medium.com/tag/anxiety/archive/2020/06/16
https://medium.com/tag/anxiety/archive/2020/06/17
https://medium.com/tag/anxiety/archive/2020/06/18
https://medium.com/tag/anxiety/archive/2020/06/19
https://medium.com/tag/anxiety/archive/2020/06/20
https://medium.com/tag/anxiety/archive/2020/06/21
https://medium.com/tag/anxiety/archive/2020/06/22
https://medium.com/tag/anxiety/archive/2020/06/23
https://medium.com/tag/anxiety/archive/2020/06/24
https://medium.com/tag/anxiety/archive/2020/06/25
https://medium.com/tag/anxiety/archive/2020/06/26
https://medium.com/tag/anxiety/archive/2020/06/27
https://medium.com/tag/anxiety/archive/2020/06/28
https://medium.com/tag/anxiety/archive/2020/06/29
https://medium.com/tag/anxiety/archive/2020/06/30
https://medium.com/tag/anxiety/archive/2020/07/01
https://medium.com/tag/anxiety/archive/2020/07/02


https://medium.com/tag/anxiety/archive/2020/11/24
https://medium.com/tag/anxiety/archive/2020/11/25
https://medium.com/tag/anxiety/archive/2020/11/26
https://medium.com/tag/anxiety/archive/2020/11/27
https://medium.com/tag/anxiety/archive/2020/11/28
https://medium.com/tag/anxiety/archive/2020/11/29
https://medium.com/tag/anxiety/archive/2020/11/30
https://medium.com/tag/anxiety/archive/2020/12/01
https://medium.com/tag/anxiety/archive/2020/12/02
https://medium.com/tag/anxiety/archive/2020/12/03
https://medium.com/tag/anxiety/archive/2020/12/04
https://medium.com/tag/anxiety/archive/2020/12/05
https://medium.com/tag/anxiety/archive/2020/12/06
https://medium.com/tag/anxiety/archive/2020/12/07
https://medium.com/tag/anxiety/archive/2020/12/08
https://medium.com/tag/anxiety/archive/2020/12/09
https://medium.com/tag/anxiety/archive/2020/12/10
https://medium.com/tag/anxiety/archive/2020/12/11
https://medium.com/tag/anxiety/archive/2020/12/12
https://medium.com/tag/anxiety/archive/2020/12/13


In [7]:
for i in range(len(data)):
    url = data[i][1]
    print(url)
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')

        sections = soup.find_all('section')
        all_paragraphs = []
        section_titles = []

        for section in sections:
            paragraphs = section.find_all('p')
            for paragraph in paragraphs:
                all_paragraphs.append(paragraph.text)

            subs = section.find_all('h1')
            for sub in subs:
                only_text = BeautifulSoup(sub.text)
                section_titles.append(only_text.get_text())
        # to ensure that we only append the text, some articles contain mins to read variable
        final_paragraphs = ""
        sep = " "
        if len(all_paragraphs) > 2:
            if "min read" in all_paragraphs[0] or "min read" in all_paragraphs[1]:
                final_paragraphs = sep.join(all_paragraphs[2:])
            else:
                final_paragraphs = sep.join(all_paragraphs[1:])
        else:
            final_paragraphs = sep.join(all_paragraphs)

        data[i].append(final_paragraphs)
    except requests.exceptions.ConnectionError as e:
        data[i].append("999999")
    
    
print(data[0])
print(data[117])

https://medium.com/@corinnasee/its-wednesday-and-i-m-dreading-going-back-to-work-401a5cd65461
https://medium.com/@pangu10/a-letter-to-myself-musings-during-a-nervous-breakdown-9fd9582cfb9
https://medium.com/@anmol.gulati9513/escapism-its-ok-to-seek-an-escape-226116c9ad62
https://medium.com/@katiefelber/my-top-10-panic-attacks-of-the-decade-5d276369eeae
https://medium.com/@thenerdyartist/saying-no-to-anxious-love-and-other-lessons-from-the-last-decade-eb286eaa7f70
https://medium.com/@mayastrong/ode-to-my-overgrown-eyebrows-802796366696
https://medium.com/take-a-breath/the-day-my-anxiety-appeared-2f01713b786a
https://medium.com/@EBKH/bienvenida-a-mi-vida-ansiedad-494d3034be37
https://medium.com/@thankyoudisco/its-just-wednesday-8507e3a724a8
https://medium.com/@dnogler/well-played-anxiety-b14fa10ecd54
https://medium.com/@robertfaldo/this-story-will-change-your-perspective-about-what-you-find-stressful-72841372f725
https://medium.com/@iamdustinlee/things-i-worried-about-in-2019-that-never-

https://medium.com/scribe/drowning-but-still-fighting-for-air-14017db8e01b
https://medium.com/@thealternativecampfire/grace-5-things-to-liberate-anxiety-from-your-life-and-start-living-freely-in-2020-49be687264f
https://medium.com/@taylorstranaghan1/5-ways-to-promote-a-healthy-gut-to-manage-your-anxiety-7ee2d83381e6
https://medium.com/@literary_device/hello-internet-its-been-a-while-8ccae7c24de
https://medium.com/house-of-haiku/panic-attack-c1b96d867ee6
https://medium.com/@marfangirl/100-days-to-overcome-depression-chronic-pain-from-marfan-syndrome-and-financial-troubles-day-34-f6baa6d2f9b1
https://medium.com/@marfangirl/100-days-to-overcome-depression-chronic-pain-from-marfan-syndrome-and-financial-troubles-day-33-ad5950164f72
https://medium.com/@malemkarina/choked-the-drowned-girl-22f310800dfa
https://medium.com/@studiofaff./three-books-that-altered-my-perspective-1406d9496fc8
https://medium.com/@marfangirl/100-days-to-overcome-depression-chronic-pain-from-marfan-syndrome-and-financi

https://medium.com/@christopherjfalvey/you-dont-need-to-be-motivated-7ec73a15b17d
https://medium.com/@shrishtisahu/framework-for-mastering-your-mind-overcoming-anxiety-9e4a46812800
https://medium.com/@nikkijameszellner/support-the-troops-support-their-families-think-before-you-share-a0b40782ed56
https://medium.com/@tj_bartel/getting-over-relationship-anxiety-1b9c999349f7
https://medium.com/@jill_45347/anxiety-makes-me-a-mean-mom-72b0bd77d5e1
https://medium.com/@theanxioustruth/what-if-my-anxietcomes-back-examining-recovery-and-the-acceptable-bubble-d36933177a03
https://medium.com/@AngelOnHiatus/theres-no-such-thing-as-happy-endings-d53883a6ebd9
https://medium.com/@Ess.D/are-you-in-that-much-of-a-hurry-e8379f44ab1
https://medium.com/@empire-of-dust/the-schedule-of-the-world-b084d887be0e
https://medium.com/@nicolenadineb/how-i-deal-with-mental-struggles-and-things-you-can-try-too-fe51427bac03
https://medium.com/@lindsey_58509/5-unexpected-symptoms-of-anxiety-you-may-never-have-heard-of-a

https://medium.com/@desabafosmeus/maybe-im-just-broken-ed6e10b3d799
https://medium.com/@plthomasedd/a-pictures-worth-18a2553a6886
https://medium.com/@lynettemagdalenalott/how-anxiety-is-triggered-5d0e66b79c47
https://medium.com/@FaceIt/internal-saboteurs-8de967282800
https://medium.com/the-landscape-of-healing/its-saturday-night-317b5fa03c39
https://medium.com/@srugers/why-you-are-not-going-to-want-to-go-to-your-first-na-aa-meeting-e2015540f679
https://medium.com/@adina_aldea/fear-of-writing-essays-64fcc9758185
https://medium.com/@pallavigupta_22207/does-god-exist-we-dont-know-can-he-help-us-he-might-3b99315dbe9f
https://medium.com/@king.heidim/not-really-o-3bd119d09dff
https://medium.com/the-mental-health-update/this-is-how-im-defeating-my-anxiety-in-2020-2363c310209e
https://medium.com/@alexanderkopke/drop-out-of-your-robot-existence-9ab80304185f
https://medium.com/@camilla.lyonnet/a-poem-for-the-worried-b548d011cfe
https://medium.com/@akankshababbar2/a-superpower-i-wish-i-had-440cd9

https://medium.com/@muzakkerafroz/reflecting-world-like-a-clear-mirror-not-like-an-opaque-one-c018c478be6d
https://medium.com/@rhsf/millennials-and-gen-zs-are-stressed-and-anxious-it-could-be-affecting-our-children-df0647cc9a27
https://medium.com/scribe/ambiguous-grief-is-real-2bdffc95554c
https://medium.com/@theleapingconnection/the-shit-will-hit-the-fan-60461055c5c5
https://medium.com/you-seem-fine/processing-trauma-honestly-2840712de50f
https://medium.com/@apops1/the-routine-of-despair-and-hope-8276d5728472
https://medium.com/@lisacostello/year-end-round-up-703f9f616f31
https://medium.com/take-a-breath/sunday-thoughts-ad95929b6bf7
https://medium.com/live-your-life-on-purpose/its-time-travel-not-overthinking-and-i-m-trying-to-stop-af7ee04ffda6
https://medium.com/@jennygarbis/pizza-is-the-most-instagrammed-food-1c324856dcf
https://medium.com/@rezenatewithbrendan/through-anxiety-and-addiction-aafcd89b918a
https://medium.com/@hinach168/why-mental-health-at-workplace-is-imperative-d05355

...

https://medium.com/the-bolt-hole/how-to-get-out-of-stress-mode-3a29111e9cf2
https://medium.com/@paulzienicke/in-praise-of-pessimism-f978ee4d2e95
https://medium.com/@mollytimkil/my-inexplicable-hatred-for-packing-38bfbf5f8eda
https://medium.com/top-level-sports/i-never-watch-my-favorite-teams-live-903a1cacec23
https://medium.com/shelterme/a-day-in-the-life-of-anxiety-arthritis-and-writing-14a6bd6b5658
https://medium.com/resistance-poetry/f-ck-you-and-2020-too-2c2c7548a653
https://medium.com/@devotable/daily-devotion-psalm-27-14-spiritual-courage-d2f6771b047c
https://medium.com/an-idea/your-plan-to-take-back-control-from-anxiety-d67a861aca93
https://medium.com/@alyssaramella/a-note-on-living-in-the-moment-1e6cb5cbe50c
https://medium.com/ipoetry/the-rivulets-632dcdb22897
https://medium.com/@cayliehausman/interlude-conversations-observations-238f247b48e8
https://medium.com/@shannonrshort/talking-about-mental-health-in-relationships-tips-and-tricks-7621f15a7ce0
https://medium.com/@grahamjon

https://medium.com/the-ascent/living-with-integrity-69ea131e2a2d
https://medium.com/invisible-illness/an-easy-breathing-exercise-to-calm-your-anxiety-effectively-f8d6de64c4de
https://medium.com/@layne88/proposing-during-a-pandemic-cd60cf5e4743
https://medium.com/@marionneubronner/sadder-fatter-and-madder-this-christmas-2020-and-how-i-am-turning-that-around-14c014cf8669
https://medium.com/@avoksh/my-pornography-addiction-how-i-recovered-8bccd6101c14
https://medium.com/illumination/music-remedy-3ffd3eeb2fab
https://medium.com/@ashley-coots/the-struggles-of-friendship-during-a-pandemic-b961d5c3296e
https://medium.com/@pixie34751/the-depression-diaries-why-iii-a440b4376949
https://medium.com/@edithvolgyi/5-things-to-do-to-lower-you-anxiety-277ad1d23492
https://medium.com/@destinyharris/anxiety-affirmations-98d082167a5
https://medium.com/@mominahambreen/coping-with-anxiety-880160822ecf
https://medium.com/@alicemcknight/my-anxious-habit-cda6803faaa3
https://medium.com/@mindsum/getting-help-f

/usr/local/lib/python3.8/site-packages/bs4/__init__.py:417: MarkupResemblesLocatorWarning: "https://www.youtube.com/watch?v=RsEZmictANA" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


https://medium.com/@shelbysmith-co/uncertainty-the-only-thing-thats-certain-about-the-future-18eb79528f85
https://medium.com/invisible-illness/choosing-between-your-money-or-your-mental-health-5d016d757de1
https://medium.com/curious/worry-is-deceitful-91-4-of-your-worries-will-not-come-true-18ec27489b06
https://medium.com/@stephenchamberlin/the-most-important-question-in-life-is-why-c704eb57b318
https://medium.com/scrittura/the-worries-9a75510c8153
https://medium.com/nameless-aimless/how-to-with-john-wilsons-abundant-heart-d51ac8cd0cc5
https://medium.com/illumination/i-tested-the-old-saying-that-a-watched-pot-never-boils-as-i-prepared-my-first-cup-of-coffee-this-1befc2acfa53
https://medium.com/@isabel-ahermosillo/you-are-too-into-your-head-c1cb00f6f268
https://medium.com/@saffdotcom/the-beauty-in-the-pain-of-feeling-too-much-56f36bd53630
https://medium.com/@katymorincoaching/how-to-beat-the-winter-blues-with-a-beam-bb3e7541d8fe
https://medium.com/scrittura/im-dying-without-poetry-2361b

https://psiloveyou.xyz/7-ways-naturally-anxious-people-can-learn-to-be-deeply-calm-74ac306d42ca
https://medium.com/@farahnuraini/redefining-confidence-584ef279713d
https://medium.com/change-becomes-you/10-tips-for-emotional-healing-7543be470c05
https://medium.com/@kmsparacio/nonlinear-recovery-4c66b0bc978d
https://medium.com/@bebadive/6-ways-i-use-the-adult-chair-model-to-heal-my-depression-36a1005ce963
https://medium.com/@beckylucywhiting/hey-7e790e0d2650
https://medium.com/@sarahzimmermanwriter/here-i-sit-in-a-strangers-kitchen-d56a46976bf9
https://medium.com/you-seem-fine/i-had-anxiety-before-it-was-cool-f73a83709aa
https://medium.com/@sobermamamtz/what-depression-isnt-cec87d5c35c9
https://medium.com/@niraj04/talking-about-anxiety-df877cd24ec8
https://medium.com/@matt-19677/how-im-learning-to-leave-my-anxiety-fuel-tank-on-empty-286057240e12
https://medium.com/@isabellatomeo/izzy-after-dark-234bff0d93e
https://medium.com/@meipoeta/poema-penh-asco-a67f590e2c6b
https://medium.com/@luci

In [8]:
medium_df = pd.DataFrame(data, columns=['id', 'url', 'title', 'date', 'text'])

In [9]:
medium_df 

,id,url,title,date,text
0,1,https://medium.com/@corinnasee/its-wednesday-a...,"It’s Wednesday, And I’m Dreading Going Back To...",2020-01-01,I like my job. But I also think I’ve never qui...
1,2,https://medium.com/@pangu10/a-letter-to-myself...,A letter to myself: musings during a nervous b...,2020-01-01,"I found this letter in amongst my files, dated..."
2,3,https://medium.com/@anmol.gulati9513/escapism-...,Escapism; It’s ok to seek an escape.,2020-01-01,This is my attempt to share one of the very wa...
3,4,https://medium.com/@katiefelber/my-top-10-pani...,My Top 10 Panic Attacks of the Decade,2020-01-01,"(1 of 10):-Summer, 2009. Birthright Israel. Ha..."
4,5,https://medium.com/@thenerdyartist/saying-no-t...,Saying NO to Anxious Love and other lessons fr...,2020-01-01,I struggled to write this because I honestly d...
...,...,...,...,...,...
12473,12474,https://medium.com/@ohmyjoshofficial/out-of-my...,Out of my depth…,2020-12-31,"Last night, before I tried to fall asleep, I s..."
12474,12475,https://medium.com/@kfranks5302/silent-screams...,"Silent Screams, a Cry for Help",2020-12-31,"More often than not, I find myself keeping my ..."
12475,12476,https://medium.com/@laurajo12/anxiety-42bbafae...,ANXIETY,2020-12-31,It happens to all of us. It lives within you a...
12476,12477,https://medium.com/@maricint369/vent-2349e77179c3,Vent,2020-12-31,"·Dec 31, 2020 The amount of fear I have for be..."


In [10]:
medium_df = medium_df.drop(medium_df.columns[[0, 1, 3]], axis=1)

In [11]:
medium_df['year'] = year
medium_df

,title,text,year
0,"It’s Wednesday, And I’m Dreading Going Back To...",I like my job. But I also think I’ve never qui...,2020
1,A letter to myself: musings during a nervous b...,"I found this letter in amongst my files, dated...",2020
2,Escapism; It’s ok to seek an escape.,This is my attempt to share one of the very wa...,2020
3,My Top 10 Panic Attacks of the Decade,"(1 of 10):-Summer, 2009. Birthright Israel. Ha...",2020
4,Saying NO to Anxious Love and other lessons fr...,I struggled to write this because I honestly d...,2020
...,...,...,...
12473,Out of my depth…,"Last night, before I tried to fall asleep, I s...",2020
12474,"Silent Screams, a Cry for Help","More often than not, I find myself keeping my ...",2020
12475,ANXIETY,It happens to all of us. It lives within you a...,2020
12476,Vent,"·Dec 31, 2020 The amount of fear I have for be...",2020


In [12]:
fname = 'medium_text_{}.csv'.format(year)
medium_df.to_csv(fname, index=False)